In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html 
from dash.dependencies import Input, Output

In [2]:
# Load Data
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

In [3]:
# Get the available levels and student groups
available_levels = assessments['school_lvl'].unique()
available_groups = ['All Students'] + assessments['student_group'].unique().tolist()

In [4]:
# Build App
app = JupyterDash(__name__)

In [5]:
# App layout
app.layout = html.Div([
    html.H1("Claytor's Stuff"),
    html.Div([
        html.Div([
            html.Label("Select Level"),
            dcc.Dropdown(
                id='level-dropdown',
                options=[{'label': level, 'value': level} for level in available_levels],
                value='Elementary',
                clearable=False
            )
        ], style={'width': '50%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Select Student Group"),
            dcc.Dropdown(
                id='group-dropdown',
                options=[{'label': group, 'value': group} for group in available_groups],
                value='All Students',
                clearable=False
            )
        ], style={'width': '50%', 'display': 'inline-block'})
    ]),
    dcc.Graph(id='graph1'),
    dcc.Graph(id='graph2')
])

In [6]:
# Define callback to update graph1 based on level and group selection
@app.callback(
    Output('graph1', 'figure'),
    [Input('level-dropdown', 'value'), Input('group-dropdown', 'value')]
)
def update_graph1(level, group):
    # Filter DataFrame based on level and group selections
    if group == 'All Students':
        filtered_data = assessments[assessments['school_lvl'] == level]
        title = f'Average Proficiency Scores of {level} Schools for All Students'
    else:
        filtered_data = assessments[(assessments['school_lvl'] == level) & (assessments['student_group'] == group)]
        title = f'Average Proficiency Scores of {level} Schools for {group} Group'

    # Calculate average pct_met_exceeded for each subject_area and year
    average_data = filtered_data.groupby(['subject_area', 'year'])['pct_met_exceeded'].mean().reset_index()

    # Create a line chart
    fig = px.line(
        average_data,
        x='year',
        y='pct_met_exceeded',
        color='subject_area',
        title=title,
        height=600,
        width=800
    )

    # Update marker settings
    fig.update_traces(mode='markers+lines', marker=dict(size=5))

    # Disconnect lines from 2020
    fig.update_traces(connectgaps=False)

    # Update legends and axis labels
    fig.update_layout(
        legend_title='Subject Area',
        xaxis_title='Year',
        yaxis_title='Average % of Students Who Met or Exceeded Expectations'
    )

    # Add footnotes
    footnote1_text = "* Testing was canceled for the 2019 - 2020 school year due to school closures."
    fig.add_annotation(
        text=footnote1_text,
        xref="paper",
        yref="paper",
        x=0,
        y=-0.16,
        showarrow=False,
        align='right',
        font=dict(size=10)
    )
    footnote2_text = "** Science content area not assessed in the 2018 - 2019 school year."
    fig.add_annotation(
        text=footnote2_text,
        xref="paper",
        yref="paper",
        x=0,
        y=-0.2,
        showarrow=False,
        align='right',
        font=dict(size=10)
    )

    return fig

In [7]:
# Define callback to update graph2 based on level and group selection
@app.callback(
    Output('graph2', 'figure'),
    [Input('level-dropdown', 'value'), Input('group-dropdown', 'value')]
)
def update_graph2(level, group):
    # Filter DataFrame based on level and group selections
    if group == 'All Students':
        filtered_data = assessments[assessments['school_lvl'] == level]
        title = f'Another Chart for {level} Schools for All Students'
    else:
        filtered_data = assessments[(assessments['school_lvl'] == level) & (assessments['student_group'] == group)]
        title = f'Another Chart for {level} Schools for {group} Group'

    # Create another chart using the filtered data

    # Replace this with your desired chart code for graph2
    fig = px.scatter(filtered_data, x='x_column', y='y_column', title=title)

    return fig

In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server(mode="jupyterlab")

Dash is running on http://127.0.0.1:8050/

